In [1]:
import pandas as pd
from datetime import datetime, timedelta
import util_calendar

In [2]:
import pygsheets
gc = pygsheets.authorize(client_secret = 'creds_google/credentials.json')

In [3]:
workbook = gc.open_by_key('1GDXzzTD1dBnXWcpjPXqfIjpLwxHtXVQtjwfLhCaZNHA')

In [4]:
study_sheet = workbook.worksheet_by_title("STUDY-PLAN")

In [5]:
# Helper Functions        
def create_event_json(series: dict) -> dict: 
    # Create a new calendar event from this event

    # Extract the Date, Time & Duration     
    start_date = datetime.strptime(series['Time']+ " " + series['Start Date'], "%I:%M %p %d-%b-%Y")
    duration = datetime.strptime(series['Duration'], "%H:%M")

    # Calculate the End Date
    end_date = start_date + timedelta(minutes = duration.minute, hours = duration.hour)

    
    timezone = '+03:00'
    
    
    # Convert to Text ISO Format
#     datetime.isoformat(start_date, timespec='seconds') + timezone
    start_date = datetime.isoformat(start_date, timespec='seconds') + timezone
    end_date = datetime.isoformat(end_date, timespec='seconds') + timezone


    

    
    
    event_bluprnt = {
      'summary': series['Name'],
      'location': 'Virtual',
      'start': { 
        'dateTime': start_date,
        'timeZone': 'GMT' + timezone
      },
      'end': {
        'dateTime': end_date,
          'timeZone': 'GMT' + timezone
      },
      
    }
    
    # Check to see if the keywords for frequency is valid
    rrule = _generate_rrule_pattern(series["Frequency"], series['End Date'])
    if rrule != "":
        event_bluprnt['recurrence'] = [rrule]
    
        
    return event_bluprnt
    
    
def initiate_event(event_bluprnt):
    # Pubish the event
    cal_code  = util_calendar.insert_event(event_bluprnt)
    return cal_code
    
def _generate_rrule_pattern(freq_code, end_date=''):
    
    # sUnday, Monday, Tuesday, Wednesday, tHursday, Friday, Saturday
    # FREQ=WEEKLY;BYDAY=SU,MO,TU,WE,TH,FR,SA
    DAY_CODES = {'U':'SU', 'M':'MO', 'T':'TU', 'W':'WE', 'H':'TH', 'F':'FR', 'S':'SA'}
    
    freq_code = freq_code.upper()
    rrule = ''
    
    if freq_code not in ["", "ONCE"]:
        if freq_code in ["WEEKLY", "DAILY"]:
            rrule = f'RRULE:FREQ={freq_code}'
        else:
            
            repet_pattern = []
            for letter in DAY_CODES:
                if letter in freq_code:
                    repet_pattern.append(DAY_CODES[letter])
                    
            if len(repet_pattern)>0:
                repet_pattern = ",".join(repet_pattern)
                rrule = f'RRULE:FREQ=WEEKLY;BYDAY={repet_pattern};INTERVAL=1'
                
    if end_date is not '':
        # Force Daily Pattern if "end date" exists
        if rrule == '':
            rrule = 'RRULE:FREQ=DAILY'
            
        # Add something like ;UNTIL=20210124T210000Z
        end_date = datetime.strptime(end_date, "%d-%b-%Y")
        end_date = datetime.strftime(end_date, "%Y%m%dT235959Z")
        rrule += f";UNTIL={end_date}"
                

    return rrule 


In [6]:
_generate_rrule_pattern("")

''

In [53]:
df = study_sheet.get_as_df(start="A", end="H")
df

,Name,Duration,Time,Start Date,End Date,Frequency,Control Action,Google Calendar Invite Code
0,Course on Philosophy,1:30,3:00 PM,,,,,
1,Morning Show,0:30,10:00 AM,23-Jan-2021,,,delete,3bv74if8moehshqiim9e99mhpg
2,Evening Show,1:30,7:00 PM,23-Jan-2021,24-Jan-2021,,delete,hqta52u0r1nt0654pngh5aefjk
3,IBM Data Science,0:25,10:00 PM,22-Jan-2021,,MTWHSU,,ndbm7nrrj64gb2mefj3ldgqmnc


In [54]:
new_df = pd.DataFrame(columns=df.columns)

for index in range(len(df)):
    event = df.loc[index]
    event_code = event['Google Calendar Invite Code']
    control_action = event['Control Action'].lower()
    
    if event_code == '':
        # Create New Event
        mandatory_entries = event[['Name', 'Duration', 'Time', 'Start Date']]
        if (mandatory_entries == "").any():
            print(f"Fields are missing for {event['Name']}")
        else:
            # Create Event
            event_bluprnt = create_event_json(event)
            print(event_bluprnt)
            event_code = initiate_event(event_bluprnt)
                  
            # Write to the new dataframe with the returned Invite Code
            event['Google Calendar Invite Code'] = event_code
                  
    else:
        if control_action == 'delete':
            util_calendar.delete_event(event_code)
            print(f"Event : {event['Name']} was deleted")
            continue
            
        if control_action == 'update':
            event_bluprnt = create_event_json(event)
            util_calendar.update_event(event_bluprnt, event_code)
            event['Control Action'] = ''

    new_df = new_df.append(event, ignore_index=True)
                  
# Keep new dataframe the same size as the old one.
for _ in range(0,len(df)-len(new_df)):
    new_df = new_df.append(pd.Series(), ignore_index=True)
    new_df.iloc[-1] = ""

# Write back to the sheet                  
study_sheet.set_dataframe(new_df, start='A2', copy_head=False)

Fields are missing for Course on Philosophy
Event : Morning Show was deleted
Event : Evening Show was deleted


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [ ]:
df_.iloc[len(df_)-1] = ""
df_